#  _Лекция 1: Языковые модели, сравнение строк_

# 4. Языковые модели

В этой части ноутбука вы посмотрите, как рвботают простейшие генеративные языковые модели, основанные на триграммах.

In [29]:
import re
from collections import defaultdict
import numpy as np

In [30]:
r_alphabet = re.compile(u'[а-яА-Я0-9\-]+|[.,:;?!]+') # clean text

#### Вспомогательные ф-ии, возвращающие генераторы с линиями  и токенами загруженного корпуса текстов

In [31]:
def gen_lines(corpus):
    data = open(corpus)
    for line in data:
        yield line.lower()

def gen_tokens(lines):
    for line in lines:
        line = re.sub('\.{2,3}', '\.', line)
        for token in r_alphabet.findall(line):
            yield token

In [32]:
print('\.{2,3}')

\.{2,3}


#### Генерация словесных триграмм

In [33]:
def gen_trigrams(tokens):
    t0, t1 = '$', '$'
    for t2 in tokens:
        yield t0, t1, t2
        if t2 in '.!?': # если это конец предложения
            yield t1, t2, '$'
            yield t2, '$','$'
            t0, t1 = '$', '$' # начало следующего предложения
        else:
            t0, t1 = t1, t2

In [34]:
for i in gen_trigrams(['ab', '.', 'cd', 'ef']):
    print (i)

('$', '$', 'ab')
('$', 'ab', '.')
('ab', '.', '$')
('.', '$', '$')
('$', '$', 'cd')
('$', 'cd', 'ef')


#### Реализуем простейшую триграммную генеративную модель

In [35]:
def pure_trigram_model(corpus):
    lines = gen_lines(corpus)
    tokens = gen_tokens(lines)
    trigrams = gen_trigrams(tokens)
    
    (bg, tg) = (defaultdict(lambda: 0.0) for i in range(2))

    for t0, t1, t2 in trigrams:
        bg[t0, t1] += 1
        tg[t0, t1, t2] += 1

    model = {}  # объект model - простейший словарь, хранящий вероятности следующего слова для каждой биграммы
    ## Пример:
    # model_tg_pure['онегин', 'был']
    # Out: [(',', 0.5), ('готов', 0.5)]
    
    for (t0, t1, t2), tg_cnt in tg.items():
        tg_prob = tg_cnt/bg[t0, t1]   #обратите внимание на оценку вероятности триграммы
        if model.get((t0, t1)):
            model[t0, t1].append((t2, tg_prob))  # (след. слово, вероятность)
        else:
            model[t0, t1] = [(t2, tg_prob)]
    return model

#### Функции, возвращающие следующий токен

In [69]:
def rand_start_token(seq):
    """Возвращает рандомный стартовый токен, сэплируемый в соответствии с вероятностью встречаемости в тексте"""
    seq_ = [x for x in seq if x[0].isalpha()]  # не будем стартовать с пунктуации
    tok_seq, probs = [x[0] for x in seq_], [x[1] for x in seq_]
    probs = [x/sum(probs) for x in probs]  # для np.random.choice сумма вероятностей должна суммироваться к 1 
                                           #(нарушается, если выбросили пунктуационные токенв)
    return np.random.choice(tok_seq, p=probs)

def rand_prob_token(seq):
    """Возвращает рандомный токен, сэплируемый в соответствии с вероятностью встречаемости в тексте"""
    tok_seq = [x[0] for x in seq]
    probs =  [x[1] for x in seq]
    probs = [x/sum(probs) for x in probs] 
    return np.random.choice(tok_seq, p=probs)

def max_prob_token(seq, equal_choice=False):
    """Возвращает максимально вероятный токен.
    equal_choice=True - возвращает рандомный максимально вероятный токен, если таких несколько"""
    if not equal_choice:   
        return sorted(seq, key=lambda x: x[1], reverse=True)[0][0]
    tok_seq = [x[0] for x in seq]
    probs =  [x[1] for x in seq]
    max_prob = max(probs)
    max_prob_tokens = [x[0] for x in seq if x[1]==max_prob]
    rnd_idx = np.random.randint(0,len(max_prob_tokens))
    return max_prob_tokens[rnd_idx]

### Генерация предложения

In [70]:
def generate_sentence(model, start_token=None, max_prob=False, equal_choice=False):
    phrase = ''
    t0, t1 = '$', '$'
    start_flag = True
    
    while True:
        if start_flag:
            if start_token:  # вручную заданный стартовый токен
                t1 = start_token.lower()
            else:
                t1 = rand_start_token(model[t0, t1])
            start_flag = False
        else:
            if max_prob:   # сэмплируем максимально вероятный токен
                t0, t1 = t1, max_prob_token(model[t0, t1], equal_choice)
            else:          # сэмплируем случайный токен из распределения вероятнсти встречаемости в тексте
                t0, t1 = t1, rand_prob_token(model[t0, t1])
            
        if t1 == '$': 
            break  # конец предложения - выход
 
        if t1 in ('.!?,;:') or t0 == '$':
            phrase += t1
        else:
            phrase += ' ' + t1
            
        if len(phrase.split( )) > 100: # для случая (max_prob=True, equal_choice=False)  - зачем это нужно? 
            break
            
    phrase = re.sub("--", '-', phrase)
    return phrase.capitalize()

## Давайте посмотрим, как это работает

Загрузим текст "Евгения Онегина"

In [71]:
corpus_path = "data/onegin.txt"

### создадим модель на основе триграмм

In [72]:
model_tg_pure = pure_trigram_model('data/onegin.txt')

### Давайте попробуем сгенерировать преддожения, взяв первый максимально вероятный токен

In [73]:
print(generate_sentence(model_tg_pure, start_token='онегин', max_prob=True, equal_choice=False))

Онегин был, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере


вот почему мы при генерации выше ограничились 100 токенами!

### Давайте попробуем брать случайный токен из максимально вероятных

In [74]:
for i in range(5):
    print(generate_sentence(model_tg_pure, start_token='онегин', max_prob=True, equal_choice=True))
    print()

Онегин, втайне усмехаясь, подходит к ольге взоры устремив, шептал: не знаю, ты нездорова; господь помилуй и спаси!

Онегин был, друзья!

Онегин, взорами сверкая, из-за стола гремя встает; все встали; он пьет одно стаканом красное вино; он пел поблеклый жизни цвет и вид в окно.

Онегин был, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, сожаленьем, по крайней мере, звук речей казался иногда нежней, и, в деревне счастлив и рогат носил бы стеганый халат; узнал бы жизнь домашним кругом я ограничить захотел; когда б надежду я имела хоть редко, хоть и заглядывал я встарь в академический словарь.

Онегин, скукой вновь гоним, близ него ее заметя, я думал уж о форме плана, и, может быть, еще чуднее: вот едет ленской на тройке чалых лошадей; давай обедать поскорей!



### Посэмплируем следующий токен согласно вероятностному распределению триграмм в тексте

In [75]:
for i in range(5):
    print(generate_sentence(model_tg_pure, start_token='татьяна', max_prob=False, equal_choice=True))
    print()

Татьяна на широкий луг всех маленьких ее подруг, она дрожала и бледнела.

Татьяна видит с трепетаньем, какою мыслью, замечаньем бывал онегин поражен, в чем все уверены давно, моей татьяне вс равно.

Татьяна с нетерпеньем, чтоб он не сделался поэтом, не отпирайтесь.

Татьяна спит.

Татьяна в оглавленье кратком находит азбучным порядком слова: бор, буря, цвет прекрасный увял на утренней заре пастух не гонит уж коров из хлева, и пустой.



### Самостоятельно реализуйте интерполяцию при оценке вероятности триграммы (см. лекцию)

Интерполированная оценка вероятности триграммы (w1,w2,w3) зависит от взвешенных слагаемых:
- собственно вероятности данной триграммы - вес $\lambda1$
- вероятности биграммы (w2,w3) - вес $\lambda2$
- вероятности токена w3 - вес $\lambda3$

Для сглаживания возьмите следующие значения:
- $\lambda1$ = 0.9 
- $\lambda2$ = 0.095
- $\lambda3$ = 0.005

In [76]:
lambda1 = 0.9
lambda2 = 0.095
lambda3 = 0.005


def interpolated_trigram_model(corpus):
    lines = gen_lines(corpus)
    tokens = gen_tokens(lines)
    
    cnt = 0
    for tok in tokens:
        cnt += 1
    
    lines = gen_lines(corpus)
    tokens = gen_tokens(lines)
    trigrams = gen_trigrams(tokens)
    (ug, bg, tg) = (defaultdict(lambda: 0.0) for i in range(3))

    for t0, t1, t2 in trigrams:
        ug[t0] += 1
        bg[t0, t1] += 1
        tg[t0, t1, t2] += 1

    model = {}
    
    for (t0, t1, t2), tg_cnt in tg.items():
        
        # just do it!
        
        tg_interpolated_prob = lambda1* tg_cnt/bg[t0, t1] +  lambda2* tg_cnt/ug[t1] + lambda3 * tg_cnt/cnt
        ############################
        
        if (t0, t1) in model:
            model[t0, t1].append((t2, tg_interpolated_prob))
        else:
            model[t0, t1] = [(t2, tg_interpolated_prob)]
    return model

# Потестируем модель с интерполированной оценкой:

In [77]:
model_tg_smooth = interpolated_trigram_model('data/onegin.txt')

In [78]:
print(generate_sentence(model_tg_pure, start_token='неосторожно', max_prob=True, equal_choice=False))

Неосторожно, быть может, на солнце иней в день троицын, когда б я был от балов без ума: верней нет места для признаний и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и


In [79]:
print(generate_sentence(model_tg_smooth, start_token='неосторожно', max_prob=True, equal_choice=False))

Неосторожно, быть может, на солнце иней в день троицын, когда б я был от балов без ума: верней нет места для признаний и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и для ногтей и


In [103]:
print(generate_sentence(model_tg_smooth, start_token='мой', max_prob=True, equal_choice=True))

Мой бедный ленской!


### Поиграйтесь с коэффициентами сглаживания и параметрами generate_sentence

In [80]:
for i in range(5):
    print(generate_sentence(model_tg_smooth, max_prob=False, equal_choice=True))

Начнем, пожалуй, - то верно б согласились вы, сны поэзии святой!
Он из чувства, и за столом сидят чудовища кругом: один в рогах с собачьей мордой, другой поэт роскошным слогом живописал нам первый снег, звездами падая на брег.
В какую бурю ощущений теперь он сердцем погружен!
Но так и быть рукой пристрастной прими собранье пестрых глав, полусмешных, полупечальных, простонародных, идеальных, небрежный плод моих забав, напев торкватовых октав!
Хоть не являла книга эта ни сладких вымыслов поэта, когда не в силах я; мне ваша искренность мила; она бежит, он говорил: дождусь ли дня?


# 5. Коллокации

#### Загрузим текст с произведениями Толстого, очистим его от тегов и разобъем на слова

In [81]:
import nltk

In [82]:
with open("data/tolstoy.txt") as f:
    content = f.read()

In [83]:
TAG_PATTERN = re.compile('<[^>]*>')
content = re.sub(TAG_PATTERN, ' ', content)
content_clean = re.sub("[^А-я]"," ", content.lower())  # only russian words
content_words = re.sub('\s+', ' ', content_clean).split()

### Выделим частотные коллокации с помощью nltk

In [84]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder_bg = nltk.BigramCollocationFinder.from_words(content_words)

In [85]:
finder_bg.apply_freq_filter(100)  # встречающиеся более, чем 100 раз в корпусе
finder_bg.nbest(bigram_measures.pmi, 10)  # 10 биграмм с максимальным PMI 

[('иван', 'ильич'),
 ('князю', 'андрею'),
 ('анна', 'михайловна'),
 ('хаджи', 'мурат'),
 ('хаджи', 'мурата'),
 ('князя', 'андрея'),
 ('тех', 'пор'),
 ('марья', 'дмитриевна'),
 ('княжна', 'марья'),
 ('первый', 'раз')]

# ! Найти три самые частотные триграммные коллокации слов, встречающиеся в корпусе content_words более чем 20 раз

In [86]:
# DO IT!
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder_tg = nltk.TrigramCollocationFinder.from_words(content_words)

In [87]:
finder_tg.apply_freq_filter(20)
finder_tg.nbest(trigram_measures.pmi, 3)

[('граф', 'илья', 'андреич'), ('до', 'сих', 'пор'), ('по', 'крайней', 'мере')]

# 6. Редакционное расстояние

### 6.1. Встроенная библиотека difflib

In [89]:
from difflib import SequenceMatcher, get_close_matches

In [90]:
s = SequenceMatcher(None, "выпьем пива", "выпьем чайку")
print(s.ratio()) # мера "похожести" двух строк (в диапазоне [0,1])

0.6956521739130435


подробнее о том, что за алгоритм работает под капотом SequenceMatcher можно почитать [тут](https://docs.python.org/3/library/difflib.html#difflib.SequenceMatcher)

In [91]:
words = ['сила', 'пилат', 'силач', 'текила', 'косила', 'игла', 'пилка', 'пита',]
query = 'пила'
get_close_matches(query, words, 3)

['пилка', 'пилат', 'сила']

# ! Сравните расстояние Левенштейна и расстояние, возвращаемое SequenceMatcher, для пар слов ("пила", "сила") и ("пила","пилка")```

In [105]:
# DO IT!
s = SequenceMatcher(None, "пила", "сила")
print(s.ratio())

s = SequenceMatcher(None, "пила", "пилка")
print(s.ratio())

0.75
0.8888888888888888


# !Найдите 3 ближайших слова по корпусу слов content_words (полученных в разделе с коллокациями) к слову "пицца" с помощью get_close_matches

In [106]:
# DO IT!
get_close_matches('пицца', content_words, 20)

['птица',
 'птица',
 'птица',
 'птица',
 'спицах',
 'принца',
 'принца',
 'принца',
 'принца',
 'принца',
 'принца',
 'певица',
 'спиц',
 'пятница',
 'пьяница',
 'пьяница',
 'птиц',
 'птиц',
 'птиц',
 'птиц']

## 6.2. Библиотека jellyfish

In [99]:
!pip install jellyfish

    100% |████████████████████████████████| 133kB 998kB/s ta 0:00:01
  Running setup.py bdist_wheel for jellyfish ... done
  Stored in directory: /Users/alexander/Library/Caches/pip/wheels/9c/6f/33/92bb9a4b4562a60ba6a80cedbab8907e48bc7a8b1f369ea0ae
Successfully built jellyfish
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


В данной библиотеке поддерживается множество алгоритмов, например:
- Расстояние Левенштейна
- Расстояние Дамеру-Левенштейна 
- Расстояние Яро-Винклера 
- Расстояние Хэмминга



In [100]:
import jellyfish

In [101]:
print(jellyfish.levenshtein_distance(u'jellyfish', u'smellyfihs'))

4


In [102]:
print(jellyfish.damerau_levenshtein_distance(u'jellyfish', u'smellyfihs'))

3


# ! Даны следующие пары слов:
  [("baba", "arab"), 
  ("contest", "toner"),
  ("eel", "lee"),
  ("martial", "marital"),
  ("monarchy", "democracy"),
  ("seatback", "backseat"),
  ("warfare", "farewell"),
  ("smoking", "hospital"),
  ("ape", "ea")]


Для каких из них расстояния Левененштейна и Дамерау-Левенштейна различаются? Почему?

In [115]:
# DO IT!
print(jellyfish.levenshtein_distance("ape", "ea"))
print(jellyfish.damerau_levenshtein_distance("ape", "ea"))

3
2


# Test

### После выполнения заданий, ответьте на вопросы [тут](https://docs.google.com/forms/d/e/1FAIpQLSe5Je_L4a9H0qWKfLynViVZfDrT-TdiDfvZ93nfIv1rn_D5aQ/viewform)